## 0. Import packages

In [437]:
import pandas as pd
import numpy as np
import sys 
from tqdm import tqdm
tqdm.pandas() 

sys.path.append("..")

from src.support_cleaning import normalize, fill_categories_forward_backward_massive, find_similar_values

## 1. Data import

In [438]:
brasil_public = pd.read_parquet("../data/concatenated_data.parquet")
brasil_public.info()
brasil_public.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1026299 entries, 0 to 134592
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   codigo_orgao_superior      995940 non-null   float64
 1   nome_orgao_superior        667093 non-null   object 
 2   codigo_orgao               1001182 non-null  float64
 3   nome_orgao                 991412 non-null   object 
 4   codigo_unidade_gestora     992718 non-null   float64
 5   nome_unidade_gestora       1006818 non-null  object 
 6   categoria_economica        1007321 non-null  object 
 7   origem_receita             987881 non-null   object 
 8   especie_receita            994372 non-null   object 
 9   detalhamento               996962 non-null   object 
 10  valor_previsto_atualizado  974984 non-null   object 
 11  valor_lancado              999880 non-null   object 
 12  valor_realizado            986834 non-null   object 
 13  percentual_realiza

,codigo_orgao_superior,nome_orgao_superior,codigo_orgao,nome_orgao,codigo_unidade_gestora,nome_unidade_gestora,categoria_economica,origem_receita,especie_receita,detalhamento,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado,data_lancamento,ano_exercicio,fichero
0,63000.0,None,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,NaN,data_2013
1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,"0,00","0,00","26666621,42","0,00",31/12/2013,2013.0,data_2013
2,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,"0,00","0,00","301251,13","0,00",31/12/2013,2013.0,data_2013
3,63000.0,None,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIV.ATIVA POR INFRAÇÃO ADMINISTRATIVA,"0,00","0,00","1855,58","0,00",31/12/2013,2013.0,data_2013
4,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",OUTRAS RESTITUICOES,"0,00","0,00","52140,68","0,00",31/12/2013,2013.0,data_2013


## 2. Data cleaning

First problems to correct as identified in exploration:
- Data types for numerical features valor_previsto_atualizado, valor_lancado, valor_realizado, percentual_realizado and datetime data_lancamento
- Missing values in nome_orgao_superior, as well as other columns, to be inferred from other columns and rows
- Duplicated values

### 2.1 Correcting data types

In [439]:
data_types_dict = {
    "codigo_orgao_superior": object,
    "codigo_orgao": object,  
    "codigo_unidade_gestora": object,      
    "valor_previsto_atualizado": float,
    "valor_lancado": float,  
    "valor_realizado": float,      
    "percentual_realizado": float,
    "data_lancamento": "datetime64[ns]"
}

#### 2.1.1 Replacing floating commas by floating point

In [440]:
for column, data_type in data_types_dict.items():
    if data_type == float:
        brasil_public[column] = brasil_public[column].str.replace(",",".")

#### 2.1.2 Correcting data types

In [441]:
brasil_public = brasil_public.astype(data_types_dict)
brasil_public.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1026299 entries, 0 to 134592
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   codigo_orgao_superior      995940 non-null   object        
 1   nome_orgao_superior        667093 non-null   object        
 2   codigo_orgao               1001182 non-null  object        
 3   nome_orgao                 991412 non-null   object        
 4   codigo_unidade_gestora     992718 non-null   object        
 5   nome_unidade_gestora       1006818 non-null  object        
 6   categoria_economica        1007321 non-null  object        
 7   origem_receita             987881 non-null   object        
 8   especie_receita            994372 non-null   object        
 9   detalhamento               996962 non-null   object        
 10  valor_previsto_atualizado  974984 non-null   float64       
 11  valor_lancado              999880 non-null 

In [442]:
brasil_public.describe().T.assign(missing_values= lambda x: brasil_public.shape[0] - x["count"]).T

,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado,data_lancamento,ano_exercicio
count,974984.0,999880.0,986834.0,1002165.0,1002468,769725.0
mean,28658142.527228,5276788.757497,24987362.481983,82.194852,2018-09-23 20:00:11.980033792,2018.225552
min,-214773743.0,-148347267246.910004,-156285934188.029999,-114552.0,2013-12-31 00:00:00,2013.0
25%,0.0,0.0,260.67,0.0,2017-04-01 00:00:00,2017.0
50%,0.0,0.0,3026.825,0.0,2018-09-03 00:00:00,2018.0
75%,0.0,0.0,40939.4175,0.0,2020-02-18 00:00:00,2020.0
max,1603521711208.0,357160677863.969971,771117711060.949951,72363772.0,2021-12-11 00:00:00,2021.0
std,3478919274.661988,817088925.702663,1418061883.509457,72336.691031,NaN,1.737972
missing_values,51315.0,26419.0,39465.0,24134.0,23831,256574.0


Now that the numerical values have been corrected, the revenue value columns can be explored. Odd things immediately struck when seeing the negative numbers as minimum values. Theoretically, revenue should always be positive and expenditures should appear in different reports, so one possible explanation for this could be that certain corrections to the same category are made.

In [443]:
brasil_public.describe(include=['O']).T.assign(missing_values= lambda x: brasil_public.shape[0] - x["count"])

,count,unique,top,freq,missing_values
codigo_orgao_superior,995940.0,25.0,26000.0,351008.0,30359.0
nome_orgao_superior,667093,25,Ministério da Educação,235281,359206
codigo_orgao,1001182.0,291.0,25000.0,116590.0,25117.0
nome_orgao,991412,287,Ministério da Economia - Unidades com vínculo ...,115437,34887
codigo_unidade_gestora,992718.0,364.0,170013.0,105008.0,33581.0
nome_unidade_gestora,1006818,356,SETORIAL ORCAMENTARIA E FINANCEIRA / ME,106170,19481
categoria_economica,1007321,5,Receitas Correntes,961527,18978
origem_receita,987881,15,Outras Receitas Correntes,310617,38418
especie_receita,994372,63,Serviços Administrativos e Comerciais Gerais,269440,31927
detalhamento,996962,1883,SERV.ADMINISTRAT.E COMERCIAIS GERAIS-PRINC.,154458,29337


### 2.1 Normalization of categories

#### 2.1.1 Addition of new category 'intra_orcamentaria'

Observing the value counts of categoria_economica it can be seen that the categories are actually 3, but a suffix 'intra-orcamentaria' makes it seem like there's 5. For that reason, it is convenient to create a new column splitting the original one.

In [444]:
brasil_public['categoria_economica'].value_counts()

categoria_economica
Receitas Correntes                           961527
Receitas de Capital                           29524
Receitas Correntes - intra-orçamentárias      15928
Sem informação                                  252
Receitas de Capital - intra-orçamentárias        90
Name: count, dtype: int64

In [445]:
brasil_public[['categoria_economica','intra_orcamentaria']] = brasil_public['categoria_economica'].str.split(" - ",expand=True)
brasil_public['intra_orcamentaria'].replace("intra-orçamentárias", "Yes",inplace=True)
brasil_public['intra_orcamentaria'].fillna("No", inplace=True)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_14160\1866350017.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  brasil_public['intra_orcamentaria'].replace("intra-orçamentárias", "Yes",inplace=True)
C:\Users\Miguel\AppData\Local\Temp\ipykernel_14160\1866350017.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [446]:
brasil_public['intra_orcamentaria']

0         No
1         No
2         No
3         No
4         No
          ..
134588    No
134589    No
134590    No
134591    No
134592    No
Name: intra_orcamentaria, Length: 1026299, dtype: object

#### 2.1.2 Correction of erratum

Find similar values is based on the Levensthein distance, that measures the number of character changes needed inside an element of a pair of strings for the pair to be equal. This can narrow down the exploration of erratum.

In [447]:
for column in brasil_public.select_dtypes("O").filter(regex='^(?!codigo)').columns.to_list():
    print(f"\n\n\n{column}")
    similar_pairs = find_similar_values(brasil_public[column], distance=2)
    print(similar_pairs)





nome_orgao_superior
[]



nome_orgao
[('Universidade Federal do Ceará', 'Universidade Federal do Pará'), ('Instituto Federal Goiano', 'Instituto Federal Baiano'), ('Instituto Federal do Ceará', 'Instituto Federal do Pará'), ('Instituto Federal do Pará', 'Instituto Federal do Paraná'), ('Universidade Federal do Paraná', 'Universidade Federal do Pará'), ('Companhia Docas do Ceará', 'Companhia Docas do Pará')]



nome_unidade_gestora
[('MAER - DIFERENCA CAMBIAL', 'MRE - DIFERENCA CAMBIAL'), ('INST.FED.DE EDUC.,CIENC.E TEC.DO RN', 'INST.FED.DE EDUC.,CIENC.E TEC.DO RS'), ('UNIVERSIDADE FEDERAL DO CEARA', 'UNIVERSIDADE FEDERAL DO PARA'), ('INST.FED.DE EDUC.,CIENC.E TEC.DO CEARA', 'INST.FED.DE EDUC.,CIENC.E TEC.DO PARA'), ('FUNDACAO UNIVERSIDADE FEDERAL DO PAMPA', 'FUNDACAO UNIVERSIDADE FEDERAL DO AMAPA'), ('UNIVERSIDADE FEDERAL DO PARANA', 'UNIVERSIDADE FEDERAL DO PARA'), ('COMPANHIA DOCAS DO CEARA', 'COMPANHIA DOCAS DO PARA'), ('COORDENACAO-GERAL DE ORCAMENTO E FINANCAS-MJ', 'COORDENACAO

Some results of visual inspection for erratum were:

- **"Outros receitas de serviços"** vs. **"outras receitas de serviços"**: "Outros" should be "outras."

- **"Receitas correntes - a classificar"** vs. **"receitas correntes a classificar"**: Hyphen inconsistency. Replace by underscore.

- **"Receita_da_industria_de_prod._farmoquimicos"** vs. **"receita_da_industria_de_prod.farmoquimicos"**: Underscore inconsistency. Replace dots by underscores and replace multiple underscores by one.

- **"Remuneração das disponibilidades do tesouro n"** vs. **"remuneração das disponibilidades do tesouro"**: "n" might refer to "nacional."

- **"Receitas_correntes___a_classificar"** vs. **"receitas_correntes_a_classificar"**: Excessive spacing. Replace multiple underscores by one.

- **"Serviço de reparação, manutenção e instalação"** vs. **"serviços de reparação, manutenção e instalação"**: Singular vs. plural inconsistency.

- **"Transferências de outras instituições pública"** vs. **"transferência de outras instituições públicas"**: Singular vs. plural inconsistency.

- **"Cont.pis/pasep-ñ opt.simp.nac-multas div.at."** vs. **"cont.pis/pasep-opt.simp.nac-multas div.at."**: Replace "ñ " by "".

- **"Contribuição para o pis/pasep-juros div.ativa"** vs. **"contribuição para o pis/pasep-juros-div.ativa"**: Replace hyphen and space with an underscore.



In [448]:
brasil_public.iloc[:,:10] = brasil_public.iloc[:,:10].map(lambda x: normalize(str(x)) if not pd.isna(x) else x)

Erratum have been already corrected in this last output, as the correction has been made in the form of modification to the support function ``normalize()``. Other specific corrections are made below:

In [449]:
replacements = (
    ("outros","outras"),
    ("tesouro_n","tesouro"),
    ("servicos","servico"),
    ("desenv","desenvol"),
)
for a, b in replacements:
    brasil_public["detalhamento"] = brasil_public["detalhamento"].str.replace(a,b)

After these corrections, categories should have rather consistent namings, altough some possibilities for inconsistency may still exist within the 'detalhamento' and 'nome_unidade_gestora' features. Due to its high cardinality and specificity of its used acronyms, complete assurance of correction is difficult and bears a low ROI on effort for the analysis, so it is considered complete.

### 2.2 Missing values

Columns in the dataset have each the following number of missing values:

In [450]:
brasil_public.isna().sum()

codigo_orgao_superior         30359
nome_orgao_superior          359206
codigo_orgao                  25117
nome_orgao                    34887
codigo_unidade_gestora        33581
nome_unidade_gestora          19481
categoria_economica           18978
origem_receita                38418
especie_receita               31927
detalhamento                  29337
valor_previsto_atualizado     51315
valor_lancado                 26419
valor_realizado               39465
percentual_realizado          24134
data_lancamento               23831
ano_exercicio                256574
fichero                           0
intra_orcamentaria                0
dtype: int64

In principle, all features with name and code (i.e. 'codigo_orgao_superior' and 'nome_orgao_superior') should have a one-to-one relationship with each other. If there are cases where not both corresponding values are missing, then it should allow for a correct filling of values based off its equivalent.

The same can be done with the seeming hierarchical features of categoria_economica, origem_receita and especie_receita, albeit more unidirectionally than with the previously mentioned pairs of features.

To end the filling of missing values topic, it is possible that monetary features 'valor_previsto_atualizado', 'valor_realizado' and 'percentual_realizado' also offer the possibility of filling in the missing values according to the formula that relates them.

Finally, on another note, the remaining missing will be evaluated to either:
- Drop the rows, in case they hold a percentage of missing that does not offer information to the analysis.
- Impute the value, very improbable and left to be evaluated at the time of the analysis. As imputing values by approximating them to a statistic or other techniques is delicate and requires of careful measure.

#### 2.2.1 Filling missing values

##### 2.2.1.1 Codigo orgao superior & nome_orgao_superior

As codigo_orgao_superior and nome_orgao_superior should bear a one to one relationship, filling the gaps with one another can be a good option. To do that, a mapping will be used, that takes a generated equivalence dictionary from the columns like so:

In [451]:
codigo_nome_orgao_superior = brasil_public[["codigo_orgao_superior","nome_orgao_superior"]].value_counts().index.to_list()
codigo_nome_orgao_superior_dict = {codigo: nome for codigo, nome  in codigo_nome_orgao_superior}
codigo_nome_orgao_superior_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_orgao_superior_dict.items()}
codigo_nome_orgao_superior_dict

{'26000_0': 'ministerio_da_educacao',
 '25000_0': 'ministerio_da_economia',
 '52000_0': 'ministerio_da_defesa',
 '22000_0': 'ministerio_da_agricultura_pecuaria_e_abastec',
 '30000_0': 'ministerio_da_justica_e_seguranca_publica',
 '39000_0': 'ministerio_da_infraestrutura',
 '36000_0': 'ministerio_da_saude',
 '32000_0': 'ministerio_de_minas_e_energia',
 '53000_0': 'ministerio_do_desenvolvimento_regional',
 '24000_0': 'ministerio_da_ciencia_tecnologia_inovacoes_',
 '41000_0': 'ministerio_das_comunicacoes',
 '44000_0': 'ministerio_do_meio_ambiente',
 '54000_0': 'ministerio_do_turismo',
 '55000_0': 'ministerio_da_cidadania',
 '20000_0': 'presidencia_da_republica',
 '63000_0': 'advocacia_geral_da_uniao',
 '35000_0': 'ministerio_das_relacoes_exteriores',
 '81000_0': 'ministerio_da_mulher_familia_e_direitos_huma',
 '38000_0': 'ministerio_do_trabalho_e_emprego',
 '49000_0': 'ministerio_do_desenvolvimento_agrario',
 '58000_0': 'ministerio_da_pesca_e_aquicultura',
 '37000_0': 'controladoria_geral

Applying the mapping through a fill na is done as per the following:

In [452]:
brasil_public['nome_orgao_superior_filled'] = brasil_public['nome_orgao_superior'].fillna(brasil_public['codigo_orgao_superior'].map(codigo_nome_orgao_superior_dict))
brasil_public['codigo_orgao_superior_filled'] = brasil_public['codigo_orgao_superior'].fillna(brasil_public['nome_orgao_superior'].map(codigo_nome_orgao_superior_dict_reverse))

Now, checking that the filling works correctly:

In [453]:
# check filled vs original missing
display(brasil_public[["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_orgao_superior"].isna() 
missing_filter_nome=  brasil_public["nome_orgao_superior"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_orgao_superior","nome_orgao_superior",'codigo_orgao_superior_filled', 'nome_orgao_superior_filled']].head()

codigo_orgao_superior            30359
nome_orgao_superior             359206
codigo_orgao_superior_filled     10545
nome_orgao_superior_filled       10545
dtype: int64

,codigo_orgao_superior,nome_orgao_superior,codigo_orgao_superior_filled,nome_orgao_superior_filled
18,NaN,controladoria_geral_da_uniao,37000_0,controladoria_geral_da_uniao
31,NaN,None,NaN,NaN
33,NaN,ministerio_da_agricultura_pecuaria_e_abastec,22000_0,ministerio_da_agricultura_pecuaria_e_abastec
44,NaN,ministerio_da_agricultura_pecuaria_e_abastec,22000_0,ministerio_da_agricultura_pecuaria_e_abastec
54,NaN,ministerio_da_agricultura_pecuaria_e_abastec,22000_0,ministerio_da_agricultura_pecuaria_e_abastec


,codigo_orgao_superior,nome_orgao_superior,codigo_orgao_superior_filled,nome_orgao_superior_filled
0,63000_0,None,63000_0,advocacia_geral_da_uniao
3,63000_0,None,63000_0,advocacia_geral_da_uniao
8,63000_0,None,63000_0,advocacia_geral_da_uniao
9,63000_0,None,63000_0,advocacia_geral_da_uniao
15,37000_0,None,37000_0,controladoria_geral_da_uniao


The check confirms that columns where 'nome_orgao_superior' was missing now have the correct name. None values are essentially the same as NaN, so they are converted as such.

Now the rest of pairs can be filled with the same technique.

##### 2.2.1.2 Codigo_orgao & nome_orgao

To make the code cleaner, the creation of the equivalences dictionary and the application of the filling function have been included in a higher support function.

In [454]:
codigo_nome_orgao = brasil_public[["codigo_orgao","nome_orgao"]].value_counts().index.to_list()
codigo_nome_orgao_dict = {codigo: nome for codigo, nome  in codigo_nome_orgao}
codigo_nome_orgao_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_orgao_dict.items()}
codigo_nome_orgao_dict

{'25000_0': 'ministerio_da_economia_unidades_com_vinculo_direto',
 '52904_0': 'fundo_do_exercito',
 '52132_0': 'comando_da_marinha_fundo_naval',
 '34902_0': 'fundo_nacional_de_cultura',
 '20701_0': 'instituto_brasileiro_do_meio_ambiente_e_dos_recursos_naturais_renovaveis',
 '22202_0': 'empresa_brasileira_de_pesquisa_agropecuaria',
 '37904_0': 'fundo_do_regime_geral_da_previdencia_social',
 '52911_0': 'fundo_aeronautico',
 '24901_0': 'fundo_nacional_de_desenvolvimento_cientifico_e_tecnologico',
 '30203_0': 'instituto_nacional_de_metrologia_qualidade_e_tecnologia',
 '41231_0': 'agencial_nacional_de_telecomunicacoes',
 '41232_0': 'fundo_de_universalizacao_dos_servicos_de_telecomunicacoes',
 '52000_0': 'ministerio_da_defesa_unidades_com_vinculo_direto',
 '36000_0': 'ministerio_da_saude_unidades_com_vinculo_direto',
 '25201_0': 'banco_central_do_brasil_orcamento_fiscal_e_seguridade_social',
 '30907_0': 'fundo_penitenciario_nacional',
 '20301_0': 'comissao_nacional_de_energia_nuclear',
 '222

In [455]:
brasil_public['nome_orgao_filled'] = brasil_public['nome_orgao'].fillna(brasil_public['codigo_orgao'].map(codigo_nome_orgao_dict))
brasil_public['codigo_orgao_filled'] = brasil_public['codigo_orgao'].fillna(brasil_public['nome_orgao'].map(codigo_nome_orgao_dict_reverse))

In [456]:
# check filled vs original missing
display(brasil_public[["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_orgao"].isna() 
missing_filter_nome=  brasil_public["nome_orgao"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_orgao","nome_orgao",'codigo_orgao_filled', 'nome_orgao_filled']].head()

codigo_orgao           25117
nome_orgao             34887
codigo_orgao_filled      905
nome_orgao_filled        905
dtype: int64

,codigo_orgao,nome_orgao,codigo_orgao_filled,nome_orgao_filled
23,NaN,ministerio_da_agricultura_pecuaria_e_abastecim...,22000_0,ministerio_da_agricultura_pecuaria_e_abastecim...
37,NaN,fundo_de_defesa_da_economia_cafeeira,22905_0,fundo_de_defesa_da_economia_cafeeira
87,NaN,ministerio_da_agricultura_pecuaria_e_abastecim...,22000_0,ministerio_da_agricultura_pecuaria_e_abastecim...
92,NaN,empresa_brasileira_de_pesquisa_agropecuaria,22202_0,empresa_brasileira_de_pesquisa_agropecuaria
118,NaN,companhia_nacional_de_abastecimento,22211_0,companhia_nacional_de_abastecimento


,codigo_orgao,nome_orgao,codigo_orgao_filled,nome_orgao_filled
7,63000_0,None,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...
31,22905_0,None,22905_0,fundo_de_defesa_da_economia_cafeeira
44,22202_0,None,22202_0,empresa_brasileira_de_pesquisa_agropecuaria
48,22202_0,None,22202_0,empresa_brasileira_de_pesquisa_agropecuaria
72,22201_0,None,22201_0,instituto_nacional_de_colonizacao_e_reforma_ag...


##### 2.2.1.3 codigo_unidade_gestora & nome_unidade_gestora

In [457]:
codigo_nome_unidade_gestora = brasil_public[["codigo_unidade_gestora","nome_unidade_gestora"]].value_counts().index.to_list()
codigo_nome_unidade_gestora_dict = {codigo: nome for codigo, nome  in codigo_nome_unidade_gestora}
codigo_nome_unidade_gestora_dict_reverse = {column2: column1 for column1, column2 in codigo_nome_unidade_gestora_dict.items()}
codigo_nome_unidade_gestora_dict

{'170013_0': 'setorial_orcamentaria_e_financeira_me',
 '167086_0': 'fundo_do_exercito',
 '673001_0': 'diretoria_de_financas_sistema_para_o_pais_mm',
 '193034_0': 'ibama_inst_brasileiro_meio_ambiente_matriz',
 '135037_0': 'embrapa_setorial',
 '340002_0': 'coord_geral_de_plan_orc_fin_e_contab_fnc',
 '513001_0': 'coordenacao_de_orcamento_e_financas_do_frgps',
 '121002_0': 'secretaria_de_econ_e_fin_da_aer_f_aer_',
 '240901_0': 'fundo_nac_de_desenv_cient_e_tecnologico',
 '183023_0': 'instituto_nac_de_metrolog_qualid_e_tecnolog',
 '413047_0': 'fundo_de_univers_dos_serv_telecomunicacoes',
 '110407_0': 'departamento_de_planej_orc_e_financas_md_',
 '413001_0': 'agencia_nacional_de_telecomunicacoes_sede',
 '250088_0': 'spo_coord_geral_de_orc_e_financas_ms',
 '173057_0': 'banco_central_do_brasil',
 '200333_0': 'depen_diretoria_executiva',
 '113209_0': 'cnen_orcamento_e_financas',
 '135100_0': 'companhia_nacional_de_abastecimento',
 '373001_0': 'dpto_de_administracao_financeira_daf_incra',
 '15317

In [458]:
brasil_public['nome_unidade_gestora_filled'] = brasil_public['nome_unidade_gestora'].fillna(brasil_public['codigo_unidade_gestora'].map(codigo_nome_unidade_gestora_dict))
brasil_public['codigo_unidade_gestora_filled'] = brasil_public['codigo_unidade_gestora'].fillna(brasil_public['nome_unidade_gestora'].map(codigo_nome_unidade_gestora_dict_reverse))

In [459]:
# check filled vs original missing
display(brasil_public[["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].isna().sum())

missing_filter_codigo = brasil_public["codigo_unidade_gestora"].isna() 
missing_filter_nome=  brasil_public["nome_unidade_gestora"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_codigo,["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].head())

# printing missing name
brasil_public.loc[missing_filter_nome,["codigo_unidade_gestora","nome_unidade_gestora",'codigo_unidade_gestora_filled', 'nome_unidade_gestora_filled']].head()

codigo_unidade_gestora           33581
nome_unidade_gestora             19481
codigo_unidade_gestora_filled      614
nome_unidade_gestora_filled        614
dtype: int64

,codigo_unidade_gestora,nome_unidade_gestora,codigo_unidade_gestora_filled,nome_unidade_gestora_filled
9,NaN,coord_geral_de_orc_fin_e_anal_cont_agu,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu
11,NaN,diretoria_de_gestao_interna_se_cgu,110174_0,diretoria_de_gestao_interna_se_cgu
35,NaN,coord_geral_de_orcamento_e_financas_mapa,130101_0,coord_geral_de_orcamento_e_financas_mapa
38,NaN,coord_geral_de_orcamento_e_financas_mapa,130101_0,coord_geral_de_orcamento_e_financas_mapa
74,NaN,None,NaN,NaN


,codigo_unidade_gestora,nome_unidade_gestora,codigo_unidade_gestora_filled,nome_unidade_gestora_filled
19,110174_0,None,110174_0,diretoria_de_gestao_interna_se_cgu
46,135100_0,None,135100_0,companhia_nacional_de_abastecimento
74,NaN,None,NaN,NaN
84,130101_0,None,130101_0,coord_geral_de_orcamento_e_financas_mapa
92,135037_0,None,135037_0,embrapa_setorial


##### 2.2.1.4 categoria_economica & origem_receita & especie_receita 

These revenues groups should share a non-shared hierarchy, meaning that a origem_receita only has a categoria_economica. If that is the case, a unidirectional filling from origem_receita to categoria_economica could be done. That could also be the case for especie_receita.

Let's check that by visually inspecting the value counts, first ordered by origem_receita with respect to categoria_economica. 

In [460]:
pd.set_option("display.max_rows",85)
brasil_public[['categoria_economica', 'origem_receita']].value_counts().reset_index().sort_values(by="origem_receita")

,categoria_economica,origem_receita,count
7,receitas_de_capital,alienacao_de_bens,11761
8,receitas_de_capital,amortizacoes_de_emprestimos,10667
4,receitas_correntes,contribuicoes,63124
3,receitas_correntes,impostos_taxas_e_contribuicoes_de_melhoria,74062
10,receitas_de_capital,operacoes_de_credito,4735
0,receitas_correntes,outras_receitas_correntes,304992
12,receitas_de_capital,outras_receitas_de_capital,1004
6,receitas_correntes,receita_agropecuaria,12938
1,receitas_correntes,receita_de_servicos,304575
5,receitas_correntes,receita_industrial,17734


As suspected, each categoria_economica bears a non-shared hierarchy with respect to origem_receita, which then means that the previous method for filling NaNs can again be used. Still, there would be just one modification to make for this to work and that is to create a separate column for receitas 'intra-orcamentárias' [Yes/no] to remove special cases. 

In [461]:
brasil_public[['categoria_economica']]

,categoria_economica
0,receitas_correntes
1,receitas_correntes
2,receitas_correntes
3,receitas_correntes
4,receitas_correntes
...,...
134588,receitas_de_capital
134589,receitas_correntes
134590,receitas_correntes
134591,receitas_correntes


In [462]:
brasil_public[['categoria_economica', 'origem_receita']].value_counts().reset_index().sort_values(by="origem_receita")

,categoria_economica,origem_receita,count
7,receitas_de_capital,alienacao_de_bens,11761
8,receitas_de_capital,amortizacoes_de_emprestimos,10667
4,receitas_correntes,contribuicoes,63124
3,receitas_correntes,impostos_taxas_e_contribuicoes_de_melhoria,74062
10,receitas_de_capital,operacoes_de_credito,4735
0,receitas_correntes,outras_receitas_correntes,304992
12,receitas_de_capital,outras_receitas_de_capital,1004
6,receitas_correntes,receita_agropecuaria,12938
1,receitas_correntes,receita_de_servicos,304575
5,receitas_correntes,receita_industrial,17734


In [463]:
categoria_economica_receita = (brasil_public[["origem_receita","categoria_economica"]]
                                            .value_counts()
                                            .index.to_list())

categoria_economica_receita_dict = {origem: categoria for origem, categoria  in categoria_economica_receita}

brasil_public['categoria_economica_filled'] = (brasil_public['categoria_economica']
                                                .fillna(brasil_public['origem_receita']
                                                .map(categoria_economica_receita_dict)))

In [464]:
brasil_public[["origem_receita","categoria_economica"]].value_counts()

origem_receita                              categoria_economica
outras_receitas_correntes                   receitas_correntes     304992
receita_de_servicos                         receitas_correntes     304575
receita_patrimonial                         receitas_correntes     154114
impostos_taxas_e_contribuicoes_de_melhoria  receitas_correntes      74062
contribuicoes                               receitas_correntes      63124
receita_industrial                          receitas_correntes      17734
receita_agropecuaria                        receitas_correntes      12938
alienacao_de_bens                           receitas_de_capital     11761
amortizacoes_de_emprestimos                 receitas_de_capital     10667
transferencias_correntes                    receitas_correntes       6012
operacoes_de_credito                        receitas_de_capital      4735
receitas_correntes_a_classificar            receitas_correntes       3243
outras_receitas_de_capital                  rece

In [465]:
categoria_economica_receita_dict

{'outras_receitas_correntes': 'receitas_correntes',
 'receita_de_servicos': 'receitas_correntes',
 'receita_patrimonial': 'receitas_correntes',
 'impostos_taxas_e_contribuicoes_de_melhoria': 'receitas_correntes',
 'contribuicoes': 'receitas_correntes',
 'receita_industrial': 'receitas_correntes',
 'receita_agropecuaria': 'receitas_correntes',
 'alienacao_de_bens': 'receitas_de_capital',
 'amortizacoes_de_emprestimos': 'receitas_de_capital',
 'transferencias_correntes': 'receitas_correntes',
 'operacoes_de_credito': 'receitas_de_capital',
 'receitas_correntes_a_classificar': 'receitas_correntes',
 'outras_receitas_de_capital': 'receitas_de_capital',
 'transferencias_de_capital': 'receitas_de_capital',
 'sem_informacao': 'sem_informacao'}

In [466]:
# check filled vs original missing
display(brasil_public[["categoria_economica", "categoria_economica_filled"]].isna().sum())

missing_filter_categoria = brasil_public["categoria_economica"].isna() 
missing_filter_origem_receita=  brasil_public["origem_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_categoria,["categoria_economica","categoria_economica_filled"]].head())


categoria_economica           18978
categoria_economica_filled      648
dtype: int64

,categoria_economica,categoria_economica_filled
22,None,receitas_de_capital
26,None,receitas_correntes
95,None,receitas_correntes
99,None,NaN
104,None,receitas_correntes


Again, the same technique can be repeated for especie receita and origem receita if the conditions apply. Let's check their unique combinations:

In [467]:
display(brasil_public[['origem_receita', 'especie_receita']].value_counts().reset_index().sort_values(by="especie_receita"))
pd.set_option("display.max_rows",20)

,origem_receita,especie_receita,count
19,alienacao_de_bens,alienacao_de_bens_imoveis,7367
50,alienacao_de_bens,alienacao_de_bens_intangiveis,27
21,alienacao_de_bens,alienacao_de_bens_moveis,4213
15,amortizacoes_de_emprestimos,amortizacoes_de_emprestimos,10612
18,outras_receitas_correntes,bens_direitos_e_valores_incorporados_ao_patr,7720
28,receita_patrimonial,cessao_de_direitos,1421
6,contribuicoes,contribuicoes_economicas,32823
46,contribuicoes,contribuicoes_para_entidades_privadas_de_serv,78
8,contribuicoes,contribuicoes_sociais,29611
12,receita_patrimonial,delegacao_de_servicos_publicos_mediante_conce,14810


It seems that the equivalence still applies, but only for those especie_receitas that do not start with "Transferências". The rule must be that, for origem_receita to be filled, especie_receita must only correspond with one origem_receita. Let's then just take the especie_receita that appear only once in the value_counts().

In [468]:
origem_especie_receita = brasil_public[['origem_receita', 'especie_receita']].value_counts().reset_index()
print(f"Full unique combinations: {origem_especie_receita.shape[0]}")
origem_especie_receita.drop_duplicates(subset="especie_receita",keep=False,inplace=True)
print(f"Unique combinations of one especie_receita: {origem_especie_receita.shape[0]}")

origem_especie_receita = origem_especie_receita[["origem_receita","especie_receita"]].values

origem_especie_receita_dict = {detalhamento: especie for especie, detalhamento  in origem_especie_receita}

brasil_public['origem_receita_filled'] = (brasil_public['origem_receita']
                                                .fillna(brasil_public['especie_receita']
                                                .map(origem_especie_receita_dict)))

Full unique combinations: 69
Unique combinations of one especie_receita: 57


In [469]:
# check filled vs original missing
display(brasil_public[["origem_receita", "origem_receita_filled"]].isna().sum())

missing_filter_origem_receita = brasil_public["origem_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_origem_receita,["origem_receita","origem_receita_filled","especie_receita"]].head())


origem_receita           38418
origem_receita_filled     1461
dtype: int64

,origem_receita,origem_receita_filled,especie_receita
91,None,receita_de_servicos,receita_de_servicos
99,None,alienacao_de_bens,alienacao_de_bens_moveis
215,None,outras_receitas_correntes,indenizacoes_restituicoes_e_ressarcimentos
259,None,receita_industrial,receitas_da_industria_de_transformacao
280,None,outras_receitas_correntes,multas_administrativas_contratuais_e_judicia


##### 2.2.1.5 especie_receita & detalhamento 

To fill the especie_receita, things get more difficult as there are many detalhamentos and that means that they might be shared among especie_receitas. Thus, a method to find the coincidences could be to find the value_counts of both features and keep the detalhamentos that appear extrictly once.

In [470]:
especie_receita_detalhamentos = brasil_public[['especie_receita', 'detalhamento']].value_counts().reset_index()
print(f"Full unique combinations: {especie_receita_detalhamentos.shape[0]}")
especie_receita_detalhamentos.drop_duplicates(subset="detalhamento",keep=False,inplace=True)
print(f"Unique combinations of one detalhamento: {especie_receita_detalhamentos.shape[0]}")

Full unique combinations: 1648
Unique combinations of one detalhamento: 1581


It seems that there are quite enough detalhamentos that are not shared among especie_receitas, which makes room for filling especie_receitas with it.

In [471]:
especie_receita_detalhamentos = especie_receita_detalhamentos[["especie_receita","detalhamento"]].values

especie_receita_detalhamentos_dict = {detalhamento: especie for especie, detalhamento  in especie_receita_detalhamentos}

brasil_public['especie_receita_filled'] = (brasil_public['especie_receita']
                                                .fillna(brasil_public['detalhamento']
                                                .map(especie_receita_detalhamentos_dict)))

# check filled vs original missing
display(brasil_public[["especie_receita", "especie_receita_filled"]].isna().sum())

missing_filter_origem_receita = brasil_public["especie_receita"].isna()

# printing missing code
display(brasil_public.loc[missing_filter_origem_receita,["especie_receita","especie_receita_filled","detalhamento"]].head())

especie_receita           31927
especie_receita_filled     4096
dtype: int64

,especie_receita,especie_receita_filled,detalhamento
75,None,NaN,outras_receitas
96,None,multas_administrativas_contratuais_e_judicia,multas_e_juros_previstos_em_contratos
104,None,exploracao_do_patrimonio_imobiliario_do_estad,arrendamentos
130,None,transferencias_de_convenios,transf_dos_estados_df_e_suas_entidades
210,None,multas_administrativas_contratuais_e_judicia,multas_e_juros_previstos_em_contratos


##### 2.2.1.6 Filling all categorical values at once

When filling gaps from one column to another, as they fill in each new pair of columns, new values appear that would have filled and could fills missings for already processed pairs. Thus, a backward-forward functional approach can be handy for to this process to be re-applied to fill-in the now fillable gaps. 

The function for that process that has been coded in the ``support_cleaning.py`` file and its called ``fill_categories_forward_backward_massive()``. One thing to note in this funciton is that equivalences are pairwise for neighboring columns, and there might be some room for gap filling between non-adjacent columns. For that reason, the operation is performed thrice, the second time being moving right one place. Modifying the function is left for future steps, as reduction of missings via this mecanism is already satisfactory.

Having checked for each column that the missing filling step is correct, the first 10 columns can directly be overwritten by its '_filled' counterparts.

In [472]:
brasil_public.iloc[:,0:10:2] = fill_categories_forward_backward_massive(brasil_public.iloc[:,0:10:2])

Full unique combinations: 291
Unique combinations of one codigo_orgao to one codigo_orgao_superior: 291
Full unique combinations: 291
Unique combinations of one codigo_orgao_superior to one codigo_orgao: 6
Full unique combinations: 364
Unique combinations of one codigo_unidade_gestora to one codigo_orgao: 364
Full unique combinations: 364
Unique combinations of one codigo_orgao to one codigo_unidade_gestora: 267
Full unique combinations: 625
Unique combinations of one categoria_economica to one codigo_unidade_gestora: 0
Full unique combinations: 625
Unique combinations of one codigo_unidade_gestora to one categoria_economica: 106
Full unique combinations: 69
Unique combinations of one especie_receita to one categoria_economica: 57
Full unique combinations: 69
Unique combinations of one categoria_economica to one especie_receita: 1
Full unique combinations: 69
Unique combinations of one categoria_economica to one especie_receita: 1
Full unique combinations: 69
Unique combinations of one

In [473]:
brasil_public.iloc[:,:10] = fill_categories_forward_backward_massive(brasil_public.iloc[:,:10])

Full unique combinations: 25
Unique combinations of one nome_orgao_superior to one codigo_orgao_superior: 25
Full unique combinations: 25
Unique combinations of one codigo_orgao_superior to one nome_orgao_superior: 25
Full unique combinations: 291
Unique combinations of one codigo_orgao to one nome_orgao_superior: 291
Full unique combinations: 291
Unique combinations of one nome_orgao_superior to one codigo_orgao: 6
Full unique combinations: 291
Unique combinations of one nome_orgao to one codigo_orgao: 283
Full unique combinations: 291
Unique combinations of one codigo_orgao to one nome_orgao: 291
Full unique combinations: 364
Unique combinations of one codigo_unidade_gestora to one nome_orgao: 364
Full unique combinations: 364
Unique combinations of one nome_orgao to one codigo_unidade_gestora: 261
Full unique combinations: 364
Unique combinations of one nome_unidade_gestora to one codigo_unidade_gestora: 348
Full unique combinations: 364
Unique combinations of one codigo_unidade_ges

In [474]:
brasil_public.isna().sum()

codigo_orgao_superior               0
nome_orgao_superior                 0
codigo_orgao                        6
nome_orgao                          0
codigo_unidade_gestora            940
                                 ... 
nome_unidade_gestora_filled       614
codigo_unidade_gestora_filled     614
categoria_economica_filled        648
origem_receita_filled            1461
especie_receita_filled           4096
Length: 27, dtype: int64

Below, a comparison between the gap-filling operation carried out "manually" and the new function. There are cases where the manual holds less missing values. The reason for this is that during the visual inspection of categories, some equivalences were incorrectly identified as unique, and thus some values were incorrectly filled.

In [475]:
for column in brasil_public.iloc[:,:9].columns:
    manual = brasil_public[f"{column}_filled"].isna().sum()
    forward_backward = brasil_public[column].isna().sum()
    print(f"{column.capitalize()}: Manual {manual} VS forward-backward {forward_backward}")

Codigo_orgao_superior: Manual 10545 VS forward-backward 0
Nome_orgao_superior: Manual 10545 VS forward-backward 0
Codigo_orgao: Manual 905 VS forward-backward 6
Nome_orgao: Manual 905 VS forward-backward 0
Codigo_unidade_gestora: Manual 614 VS forward-backward 940
Nome_unidade_gestora: Manual 614 VS forward-backward 147
Categoria_economica: Manual 648 VS forward-backward 11
Origem_receita: Manual 1461 VS forward-backward 341
Especie_receita: Manual 4096 VS forward-backward 4084


Finally, "_filled" columns are dropped.

In [476]:
brasil_public = brasil_public.iloc[:,:-9]
brasil_public

,codigo_orgao_superior,nome_orgao_superior,codigo_orgao,nome_orgao,codigo_unidade_gestora,nome_unidade_gestora,categoria_economica,origem_receita,especie_receita,detalhamento,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado,data_lancamento,ano_exercicio,fichero,intra_orcamentaria
0,63000_0,advocacia_geral_da_uniao,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu,receitas_correntes,outras_receitas_correntes,bens_direitos_e_valores_incorporados_ao_patr,rec_divida_ativa_nao_tributaria_de_outras_rec,0.0,0.0,1297.13,0.0,2013-12-31,NaN,data_2013,No
1,63000_0,advocacia_geral_da_uniao,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu,receitas_correntes,outras_receitas_correntes,indenizacoes_restituicoes_e_ressarcimentos,recuperacao_de_despesas_de_exerc_anteriores,0.0,0.0,26666621.42,0.0,2013-12-31,2013.0,data_2013,No
2,63000_0,advocacia_geral_da_uniao,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu,receitas_correntes,outras_receitas_correntes,multas_administrativas_contratuais_e_judicia,outras_multas_e_juros_de_mora,0.0,0.0,301251.13,0.0,2013-12-31,2013.0,data_2013,No
3,63000_0,advocacia_geral_da_uniao,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu,receitas_correntes,outras_receitas_correntes,bens_direitos_e_valores_incorporados_ao_patr,rec_div_ativa_por_infracao_administrativa,0.0,0.0,1855.58,0.0,2013-12-31,2013.0,data_2013,No
4,63000_0,advocacia_geral_da_uniao,63000_0,advocacia_geral_da_uniao_unidades_com_vinculo_...,110060_0,coord_geral_de_orc_fin_e_anal_cont_agu,receitas_correntes,outras_receitas_correntes,indenizacoes_restituicoes_e_ressarcimentos,outras_restituicoes,0.0,0.0,52140.68,0.0,2013-12-31,2013.0,data_2013,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134588,20000_0,presidencia_da_republica,24208_0,instituto_nacional_de_tecnologia_da_informacao,243001_0,instituto_nac_de_tecnologia_da_informacao_iti,receitas_de_capital,operacoes_de_credito,operacoes_de_credito_mercado_interno,titulos_de_respons_tes_nac_merc_int_princ_,16940891.0,0.0,0.00,0.0,2021-04-23,2021.0,data_2021,No
134589,20000_0,presidencia_da_republica,24208_0,instituto_nacional_de_tecnologia_da_informacao,243001_0,instituto_nac_de_tecnologia_da_informacao_iti,receitas_correntes,receita_de_servicos,servicos_administrativos_e_comerciais_gerais,serv_de_regist_certif_e_fiscaliz_principal,0.0,0.0,NaN,0.0,2021-11-22,2021.0,data_2021,No
134590,20000_0,presidencia_da_republica,24208_0,instituto_nacional_de_tecnologia_da_informacao,243001_0,instituto_nac_de_tecnologia_da_informacao_iti,receitas_correntes,receita_de_servicos,servicos_administrativos_e_comerciais_gerais,serv_de_regist_certif_e_fiscaliz_principal,200000.0,0.0,0.00,0.0,2021-04-23,2021.0,data_2021,No
134591,20000_0,presidencia_da_republica,24208_0,instituto_nacional_de_tecnologia_da_informacao,243001_0,instituto_nac_de_tecnologia_da_informacao_iti,receitas_correntes,receita_de_servicos,servicos_administrativos_e_comerciais_gerais,serv_de_regist_certif_e_fiscaliz_principal,0.0,0.0,1000000.00,NaN,2021-10-05,2021.0,data_2021,No


##### 2.2.1.7 ano_exercicio

In [477]:
brasil_public.loc[brasil_public["ano_exercicio"].isna(),"ano_exercicio"] = (brasil_public
                                                                            .loc[brasil_public["ano_exercicio"].isna(),"fichero"]
                                                                            .str[-4:]).astype(int)

##### 2.2.1.8 valor_previsto_atualizado, valor_lancado, valor_realizado, percentual_realizado

In [478]:
brasil_public[['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].describe().T.assign(missing_values = lambda x: brasil_public.shape[0] - x["count"] )

,count,mean,std,min,25%,50%,75%,max,missing_values
valor_previsto_atualizado,974984.0,2.865814e+07,3.478919e+09,-2.147737e+08,0.00,0.000,0.0000,1.603522e+12,51315.0
valor_lancado,999880.0,5.276789e+06,8.170889e+08,-1.483473e+11,0.00,0.000,0.0000,3.571607e+11,26419.0
valor_realizado,986834.0,2.498736e+07,1.418062e+09,-1.562859e+11,260.67,3026.825,40939.4175,7.711177e+11,39465.0
percentual_realizado,1002165.0,8.219485e+01,7.233669e+04,-1.145520e+05,0.00,0.000,0.0000,7.236377e+07,24134.0


In [479]:
valor_previsto_atualizado_not_null = ~brasil_public['valor_previsto_atualizado'].isna()
valor_lancado_not_null = ~brasil_public['valor_lancado'].isna()
valor_realizado_not_null = ~brasil_public['valor_realizado'].isna()
percentual_realizado_not_null = ~brasil_public['percentual_realizado'].isna()


brasil_public.loc[valor_previsto_atualizado_not_null,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].query("valor_previsto_atualizado != 0")

,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado
14,1800000.0,0.0,0.00,0.0
19,403603.0,0.0,0.00,0.0
20,640279.0,0.0,985939.65,NaN
21,358413.0,0.0,194307.50,54.0
22,111097.0,0.0,68323.99,NaN
...,...,...,...,...
134408,97926843.0,0.0,349087.72,0.0
134577,28274054.0,0.0,0.00,0.0
134587,10247198.0,0.0,0.00,0.0
134588,16940891.0,0.0,0.00,0.0


There are null values for percentual_realizado that could be calculated from valor previsto. Also, which is more strange, there are values worth zero for this same feature that should worth something, based on the values present for valor_previsto_atualizado and valor_realizado.

In [480]:
percentual_realizado_not_null_or_zero = brasil_public['percentual_realizado'].isna() | brasil_public['percentual_realizado'] == 0
brasil_public.loc[percentual_realizado_not_null_or_zero,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']]

,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado
0,0.0,0.0,1297.13,0.0
1,0.0,0.0,26666621.42,0.0
2,0.0,0.0,301251.13,0.0
3,0.0,0.0,1855.58,0.0
4,0.0,0.0,52140.68,0.0
...,...,...,...,...
134587,10247198.0,0.0,0.00,0.0
134588,16940891.0,0.0,0.00,0.0
134589,0.0,0.0,NaN,0.0
134590,200000.0,0.0,0.00,0.0


In [481]:
percentual_realizado_null_or_zero = (brasil_public['percentual_realizado'].isna()) | (brasil_public['percentual_realizado'] == 0)
valor_previsto_atualizado_not_null_nor_zero = (~brasil_public['valor_previsto_atualizado'].isna()) & brasil_public['valor_previsto_atualizado'] != 0
valor_realizado_not_null= ~brasil_public['valor_realizado'].isna()

filters = percentual_realizado_null_or_zero & valor_previsto_atualizado_not_null_nor_zero & valor_realizado_not_null

brasil_public.loc[filters,['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']]

,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado
14,1800000.0,0.0,0.00,0.0
19,403603.0,0.0,0.00,0.0
20,640279.0,0.0,985939.65,NaN
22,111097.0,0.0,68323.99,NaN
24,720000.0,0.0,0.00,0.0
...,...,...,...,...
134408,97926843.0,0.0,349087.72,0.0
134577,28274054.0,0.0,0.00,0.0
134587,10247198.0,0.0,0.00,0.0
134588,16940891.0,0.0,0.00,0.0


Filling or correcting 'percentual_realizado':

In [482]:
percentual_realizado_null_or_zero = (brasil_public['percentual_realizado'].isna()) | (brasil_public['percentual_realizado'] == 0)
valor_previsto_atualizado_not_null_nor_zero = (~brasil_public['valor_previsto_atualizado'].isna()) & (brasil_public['valor_previsto_atualizado'] != 0)
valor_realizado_not_null= ~brasil_public['valor_realizado'].isna()

filters = percentual_realizado_null_or_zero & valor_previsto_atualizado_not_null_nor_zero & valor_realizado_not_null

brasil_public.loc[filters,'percentual_realizado'] = ((brasil_public.loc[filters,'valor_realizado'] / 
                                                    brasil_public.loc[filters,'valor_previsto_atualizado']) * 100)

Filling or correcting 'valor_realizado':

In [483]:
valor_realizado_null_or_zero = (brasil_public['valor_realizado'].isna()) | (brasil_public['valor_realizado'] == 0)

percentual_realizado_not_null_or_zero = (~brasil_public['percentual_realizado'].isna()) & (brasil_public['percentual_realizado'] != 0)
valor_previsto_atualizado_not_null = ~brasil_public['valor_previsto_atualizado'].isna()
 
filters = valor_realizado_null_or_zero & percentual_realizado_not_null_or_zero & valor_previsto_atualizado_not_null 

brasil_public.loc[filters,'valor_realizado'] = ((brasil_public.loc[filters,'percentual_realizado'] / 100) * 
                                                brasil_public.loc[filters,'valor_previsto_atualizado'])

Filling or correcting 'valor_previsto_atualizado':

In [484]:
valor_previsto_atualizado_null_or_zero = (brasil_public['valor_previsto_atualizado'].isna()) | (brasil_public['valor_previsto_atualizado'] == 0)

valor_realizado_not_null_nor_zero = ~brasil_public['valor_realizado'].isna()
percentual_realizado_not_null_or_zero = (~brasil_public['percentual_realizado'].isna()) & (brasil_public['percentual_realizado'] != 0)


filters = valor_previsto_atualizado_null_or_zero & valor_realizado_not_null_nor_zero & percentual_realizado_not_null_or_zero 
brasil_public.loc[filters,'valor_previsto_atualizado'] = ((brasil_public.loc[filters,'valor_realizado'] / 
                                                           brasil_public.loc[filters,'percentual_realizado']) * 100)


In [485]:
brasil_public[['valor_previsto_atualizado', 'valor_lancado',
       'valor_realizado', 'percentual_realizado']].describe().T.assign(missing_values = lambda x: brasil_public.shape[0] - x["count"] )

,count,mean,std,min,25%,50%,75%,max,missing_values
valor_previsto_atualizado,975346.0,2.879274e+07,3.478657e+09,-2.147737e+08,0.0000,0.00,0.000,1.603522e+12,50953.0
valor_lancado,999880.0,5.276789e+06,8.170889e+08,-1.483473e+11,0.0000,0.00,0.000,3.571607e+11,26419.0
valor_realizado,987108.0,2.524281e+07,1.431350e+09,-1.562859e+11,260.9475,3030.39,41006.045,7.711177e+11,39191.0
percentual_realizado,1002687.0,8.220136e+01,7.231786e+04,-1.145520e+05,0.0000,0.00,0.000,7.236377e+07,23612.0


### 2.2.2 Imputation of missing values

Missing values in the monetary features are left to stay as they are. For a couple of reasons:
- Imputing NaN values with influence the cyphers for any given category or organism, and the purpose of this analysis is pure observation and description, for the time being
- The presence of NaN values themselves may present a pattern or provide information taht could potentially lead to corruption uncovering, for example.
- They do not harm the analysis if taken into consideration the count rather than the size of the dataset for statistic calculation

### 2.2.3 Drop of high percentage null values rows

If certain rows have a high frequency of null, above 50%, it is worth dropping them as those rows provide no real information. As it can be observed, however, after filling the null values no row fulfils that condition. As said above, if all other columns are not empty, all empty 'valor' columns can still be kept for analysis.

In [501]:
(brasil_public[].isna().sum(axis=1) / brasil_public.shape[1]).max()

np.float64(0.2222222222222222)

### 2.3 Duplicates

After having filled in duplicates, it is time to drop the duplicates before saving the cleaned dataset.

In [486]:
brasil_public.duplicated().sum()

np.int64(220)

In [487]:
brasil_public.drop_duplicates(inplace=True)
brasil_public.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1026079 entries, 0 to 134592
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   codigo_orgao_superior      1026079 non-null  object        
 1   nome_orgao_superior        1026079 non-null  object        
 2   codigo_orgao               1026073 non-null  object        
 3   nome_orgao                 1026079 non-null  object        
 4   codigo_unidade_gestora     1025139 non-null  object        
 5   nome_unidade_gestora       1025932 non-null  object        
 6   categoria_economica        1026068 non-null  object        
 7   origem_receita             1025738 non-null  object        
 8   especie_receita            1021995 non-null  object        
 9   detalhamento               996881 non-null   object        
 10  valor_previsto_atualizado  975126 non-null   float64       
 11  valor_lancado              999660 non-null 

## 3. Export clean data

Now cleaned data is exported to be imported back into the exploration notebook ``exploration.ipynb`` to continue the analysis.

In [488]:
brasil_public.reset_index(drop=True).to_parquet("../data/cleaned_data.parquet")